In [4]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

# Open a CSV file for writing
with open("books.csv", "w", newline="") as csvfile:
    # Create a CSV writer
    writer = csv.DictWriter(csvfile, fieldnames=["title", "price", "rating", "image", "availability"])

    # Write the header row
    writer.writeheader()

    # Create a web driver
    driver = webdriver.Chrome()

    # Set the base URL of the website
    base_url = "https://books.toscrape.com/catalogue/page-"

    # Initialize a page number
    page_num = 1

    # Set the URL of the first page
    url = base_url + str(page_num) + ".html"

    # Initialize a flag to control the loop
    more_pages = True

    while more_pages:
        # Load the page
        driver.get(url)

        # Wait for the page to load
        driver.implicitly_wait(10)

        # Scroll down to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for the page to load after scrolling
        driver.implicitly_wait(10)

        # Parse the HTML content of the page
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Find all the books on the page
        book_tags = soup.find_all("article")

        # For each book, extract the title, price, rating, image URL, and availability
        for book_tag in book_tags:
            title = book_tag.find("h3").find("a")["title"]
            price = book_tag.find("p", class_="price_color").text[1:]  # Remove the first character from the price
            rating_class = book_tag.find("p", class_="star-rating")["class"][1]
            if rating_class == "One":
                rating = 1
            elif rating_class == "Two":
                rating = 2
            elif rating_class == "Three":
                rating = 3
            elif rating_class == "Four":
                rating = 4
            elif rating_class == "Five":
                rating = 5
            else:
                rating = 0
            image = book_tag.find("img")["src"]
            availability_tag = book_tag.find("p", class_="instock availability")
            availability_strip = availability_tag.text
            availability_strip = availability_strip.strip()
            if availability_strip == "In stock":
                availability = 1
            else:
                availability = 0

            # Store the book information in a dictionary
            book = {
                "title": title,
                "price": price,
                "rating": rating,
                "image": image,
                "availability": availability
            }

            # Write the book to the CSV file
            writer.writerow(book)

        # Find the link to the next page
        next_page_tag = soup.find("li", class_="next")
        if next_page_tag:
            url = base_url + str(page_num) + ".html"
            page_num += 1
        else:
            more_pages = False

    # Close the web driver
    driver.close()

In [3]:
import csv

# Initialize variables to store the data
titles = []
prices = []
ratings = []

# Open the CSV file and read the data
with open("books.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        titles.append(row["title"])
        prices.append(float(row["price"]))
        ratings.append(int(row["rating"]))

# Calculate the average price of the books
average_price = sum(prices) / len(prices)
print("Average price:", average_price)

# Calculate the average rating of the books
average_rating = sum(ratings) / len(ratings)
print("Average rating:", average_rating)

# Find the most expensive book
most_expensive_book = max(prices)
print("Most expensive book:", titles[prices.index(most_expensive_book)])

# Find the book with the highest rating
highest_rated_book = max(ratings)
print("Highest rated book:", titles[ratings.index(highest_rated_book)])

Average price: 35.1287450980392
Average rating: 2.9215686274509802
Most expensive book: The Perfect Play (Play by Play #1)
Highest rated book: Sapiens: A Brief History of Humankind
